In [1]:
import camelot

def extract_tables_from_pdf_camelot(pdf_path):
    """Извлекает таблицы из PDF-файла, используя camelot.

    Args:
        pdf_path (str): Путь к PDF-файлу.

    Returns:
        list: Список объектов Table из camelot.
    """
    try:
        tables = camelot.read_pdf(pdf_path, pages='all')
        return tables
    except FileNotFoundError:
        print(f"Ошибка: Файл не найден по пути '{pdf_path}'.")
        return []
    except Exception as e:
        print(f"Произошла ошибка при извлечении таблиц: {e}")
        return []

if __name__ == "__main__":
    pdf_file_path = '2016.pdf'  # Замените на путь к вашему PDF-файлу
    extracted_tables = extract_tables_from_pdf_camelot(pdf_file_path)

    # if extracted_tables:
    #     for i, table in enumerate(extracted_tables):
    #         print(f"--- Таблица № {i+1} ---")
    #         # print(table.df)  # Получаем DataFrame из объекта Table
    #         # Вы можете сохранить таблицу в CSV-файл:
    #         # table.to_csv(f'camelot_table_{i+1}.csv', index=False, encoding='utf-8')
    # else:
    #     print("Таблицы не найдены в PDF-файле.")

C:\Users\valeriyk1402\AppData\Roaming\Python\Python39\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
C

In [62]:
import pandas as pd

df = extracted_tables[0].df

columns = [
    'Model Number',
    'Item Description',
    'Detail Description',
    'Overall Width',
    'Overall Depth',
    'Overall Height',
    'Arm Height',
    'Seat Height',
    'Seat Depth',
    'Inside Width',
    'Weight (Wrap)',
    'Weight (Carton)',
    'Cubic Feet',
    'COM Price',
    'COM Yardage',
    'Year'
]

df_answer = pd.DataFrame([], columns=columns)

for page in extracted_tables:
    df = page.df
    print(page)
    for i in range(3, len(df.columns)):
        print(i)
        row_list = [
            df[i].iloc[0],
            ', '.join(df[i].iloc[1].split('\n')),
            ', '.join(df[i].iloc[2].split('\n')),
            df[i].iloc[3],
            df[i].iloc[4],
            df[i].iloc[5],
            df[i].iloc[6],
            df[i].iloc[7],
            df[i].iloc[8],
            df[i].iloc[9]
        ]
        wrap_carton_cube = df[i].iloc[11].split()
        try:
            wrap_carton_cube.remove('/')
        except:
            pass
        wrap_carton_cube = [i.strip('/') for i in wrap_carton_cube]
        if len(wrap_carton_cube) == 0:
            row_list.append('')
            row_list.append('')
            row_list.append('')
        elif len(wrap_carton_cube) != 3:
            row_list.append('')
            row_list.append('')
            row_list.append(wrap_carton_cube[0])
        else:
            row_list.append(wrap_carton_cube[0])
            row_list.append(wrap_carton_cube[1])
            row_list.append(wrap_carton_cube[2])
        row_list.append(df[i].iloc[16])
        row_list.append(df[i].iloc[31])
        row_list.append(pdf_file_path.split('.')[0])
        # print('\n'.join(row_list))

        df_answer = pd.concat([df_answer, pd.DataFrame([row_list], columns=columns)])

<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(50, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(51, 9)>
3
4
5
6
7
8
<Table shape=(

In [51]:
df_answer.reset_index(drop=True, inplace=True)

In [61]:
df_answer[df_answer['COM Price'] != ''].to_csv('part_job.csv', index=False)

In [39]:
a = '105 / 105 20.95'
a = a.split()
try:
    a.remove('/')
except:
    pass
a


['105', '105', '20.95']